# Курс "Python для анализа данных"

---
#6.4 Итоговый вариант обработчика данных. Встраивание в процесс обучения модели машинного обучения

## Где мы сейчас?


<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <ul>
    <li>1. Введение в анализ данных и разработку на языке Python </li> 
    <li>2. Основы языка </li> 
    <li>3. Библиотеки NumPy и SciPy для решения задач анализа данных и машинного обучения </li> 
    <li>4. Библиотека Pandas для работы с высокоуровневыми структурами данных </li> 
    <li>5. Визуализации результатов анализа данных с использованием библиотек Matplotlib, Seaborn, Pandas </li>
    <li><strong>6. Роль и место библиотек языка Python в построении конвейера для обработки и анализа данных. Пример обработки известного датасета</strong> <i><- Вот в этой главе!</i></li>
     <ul>
      <li>6.1 Основы построения конвейера для обработки данных</li>
      <li>6.2 Библиотека Pandas для загрузки и первичной обработки исходных данных</li>
      <li>6.3 Статистический анализ данных, получение корреляционных значений и отбор значимых признаков</li>
      <li><strong>6.4 Итоговый вариант обработчика данных. Встраивание в процесс обучения модели машинного обучения</strong><i><- Вот в этом параграфе!</i></li>
     </ul>
    </li>   
  </ul>
 </body>
</html>

## О чем будем говорить?


О предобработке данных датасета ```Titanic``` с их последующей загрузкой в классификатор на основе алгоритма машинного обучения.

## Ключевые слова, понятия и выражения






*   ```Titanic```

## Материал

### Импорт и чтение данных

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files

# Загрузка json файла для работы с kaggle.com
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"olegkok","key":"adc98f364f050d1a10f93506a68ad8a9"}'}

In [ ]:
# Расположение файла для аутентификации на kaggle.com
!mkdir -p ~/.kaggle
!mv -i kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Загрузка данных
!kaggle competitions download -c titanic
!ls

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 79.7MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 54.7MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.60MB/s]
gender_submission.csv  sample_data  test.csv  train.csv


In [ ]:
# Чтение данных
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# Считывание целевой переменной в отдельный массив
len_train_data = len(train_data)
y = train_data['Survived'].values
print('Количество позиций для обучения:', len_train_data)
print(y[:10])

Количество позиций для обучения: 891
[0 1 1 1 0 0 0 0 1 1]


In [ ]:
# Переиндексируем данные и удаляем некоторые столбцы
train_data.index = train_data.PassengerId
del train_data['PassengerId']
del train_data['Survived']

test_data.index = test_data.PassengerId
del test_data['PassengerId']

train_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# Сливаем данные в один датасет для обработки
all_data = pd.concat([train_data, test_data])
print('Размер полученных данных:', all_data.shape)
all_data.head()

Размер полученных данных: (1309, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Предобработка некоторых столбцов данных

In [ ]:
# Наличие номера кабины
all_data['Has_Cabin'] = all_data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Размер семьи
all_data['FamilySize'] = all_data['SibSp'] + all_data['Parch'] + 1

# Признак одинокого плавания
all_data['Is_alone'] = all_data["FamilySize"].apply(lambda x: 0 if x > 1 else 1)

In [ ]:
all_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Has_Cabin,FamilySize,Is_alone
PassengerId,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,2,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,2,0
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,2,0
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1,1


In [ ]:
# Использование регулярных выражений для поиска сокращений нейтральных обращений
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

# Применение к столбцу Name
all_data['Title'] = all_data['Name'].apply(get_title)
all_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Has_Cabin,FamilySize,Is_alone,Title
PassengerId,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,2,0,Mr
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,2,0,Mrs
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,1,Miss
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,2,0,Mrs
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1,1,Mr


In [ ]:
# Просмотр результатов предобработки
all_data['Title'].value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Ms            2
Major         2
Don           1
Dona          1
Lady          1
Mme           1
Sir           1
Countess      1
Jonkheer      1
Capt          1
Name: Title, dtype: int64

In [ ]:
# Исправление опечаток и группировака редких обращений
all_data['Title'] = all_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
all_data['Title'] = all_data['Title'].replace('Mlle', 'Miss')
all_data['Title'] = all_data['Title'].replace('Ms', 'Miss')
all_data['Title'] = all_data['Title'].replace('Mme', 'Mrs')

In [ ]:
all_data.head()

In [ ]:
# Очистка данных от обработанных столбцов
all_data = all_data.drop(['Name','Ticket','Cabin'], axis = 1)

In [ ]:
all_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin,FamilySize,Is_alone,Title
PassengerId,,,,,,,,,,,
1,3,male,22.0,1,0,7.2500,S,0,2,0,Mr
2,1,female,38.0,1,0,71.2833,C,1,2,0,Mrs
3,3,female,26.0,0,0,7.9250,S,0,1,1,Miss
4,1,female,35.0,1,0,53.1000,S,1,2,0,Mrs
5,3,male,35.0,0,0,8.0500,S,0,1,1,Mr


### Перекодировка некоторых категориальных значений

In [ ]:
# Перевод в числовой вид Title
title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
all_data['Title'] = all_data['Title'].map(title_mapping)
all_data['Title'] = all_data['Title'].fillna(0)

In [ ]:
# Перевод в числовой вид Sex
map_dict_sex = {'male': 1 , 'female': 0}

all_data['Sex'] = all_data['Sex'].map(map_dict_sex)

In [ ]:
# Перевод в числовой вид Embarked
map_dict_embarked = {'S': 0 , 'C': 1, 'Q':2}

all_data['Embarked'] = all_data['Embarked'].map(map_dict_embarked)


In [ ]:
all_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin,FamilySize,Is_alone,Title
PassengerId,,,,,,,,,,,
1,3,1,22.0,1,0,7.2500,0.0,0,2,0,1
2,1,0,38.0,1,0,71.2833,1.0,1,2,0,3
3,3,0,26.0,0,0,7.9250,0.0,0,1,1,4
4,1,0,35.0,1,0,53.1000,0.0,1,2,0,3
5,3,1,35.0,0,0,8.0500,0.0,0,1,1,1


In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
Pclass        1309 non-null int64
Sex           1309 non-null int64
Age           1046 non-null float64
SibSp         1309 non-null int64
Parch         1309 non-null int64
Fare          1308 non-null float64
Embarked      1307 non-null float64
Has_Cabin     1309 non-null int64
FamilySize    1309 non-null int64
Is_alone      1309 non-null int64
Title         1309 non-null int64
dtypes: float64(3), int64(8)
memory usage: 122.7 KB


In [ ]:
# Избавление от редких пропусков
all_data['Fare']  = all_data['Fare'].fillna(value = all_data['Fare'].median())
all_data['Embarked']  = all_data['Embarked'].fillna(value = 0)

In [ ]:
# Среднее арифметическое для возраста
all_data['Age']  = all_data['Age'].fillna(value = all_data['Age'].mean())

In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
Pclass        1309 non-null int64
Sex           1309 non-null int64
Age           1309 non-null float64
SibSp         1309 non-null int64
Parch         1309 non-null int64
Fare          1309 non-null float64
Embarked      1309 non-null float64
Has_Cabin     1309 non-null int64
FamilySize    1309 non-null int64
Is_alone      1309 non-null int64
Title         1309 non-null int64
dtypes: float64(3), int64(8)
memory usage: 122.7 KB


In [ ]:
all_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin,FamilySize,Is_alone,Title
PassengerId,,,,,,,,,,,
1,3,1,22.0,1,0,7.2500,0.0,0,2,0,1
2,1,0,38.0,1,0,71.2833,1.0,1,2,0,3
3,3,0,26.0,0,0,7.9250,0.0,0,1,1,4
4,1,0,35.0,1,0,53.1000,0.0,1,2,0,3
5,3,1,35.0,0,0,8.0500,0.0,0,1,1,1


###Дополнительная предобработка данных

In [ ]:
all_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin,FamilySize,Is_alone,Title
PassengerId,,,,,,,,,,,
1,3,1,22.0,1,0,7.2500,0.0,0,2,0,1
2,1,0,38.0,1,0,71.2833,1.0,1,2,0,3
3,3,0,26.0,0,0,7.9250,0.0,0,1,1,4
4,1,0,35.0,1,0,53.1000,0.0,1,2,0,3
5,3,1,35.0,0,0,8.0500,0.0,0,1,1,1


In [ ]:
# Добавим названия категориальных столбцов
categorical_columns_names = ['Pclass', 'Sex', 
                             'SibSp', 'Parch',
                             'Embarked', 'Has_Cabin',
                             'FamilySize', 'Is_alone',
                             'Title']

In [ ]:
# Кодирование категориальных данных по принципу One-Hot
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)
encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform(all_data[categorical_columns_names]),
                                           index=all_data.index)
encoded_categorical_columns.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
PassengerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
# Совмещаем закодированные категориальные признаки с остальными данными
res_data_all = all_data.join(encoded_categorical_columns)
res_data_all.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin,FamilySize,Is_alone,Title,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
PassengerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3,1,22.0,1,0,7.2500,0.0,0,2,0,1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,0,38.0,1,0,71.2833,1.0,1,2,0,3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,0,26.0,0,0,7.9250,0.0,0,1,1,4,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1,0,35.0,1,0,53.1000,0.0,1,2,0,3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,3,1,35.0,0,0,8.0500,0.0,0,1,1,1,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
categorical_columns_names

['Pclass',
 'Sex',
 'SibSp',
 'Parch',
 'Embarked',
 'Has_Cabin',
 'FamilySize',
 'Is_alone',
 'Title']

In [ ]:
# Уберем категориальные столбцы, которые были закодированы
res_data_all = res_data_all.drop(categorical_columns_names, axis=1)
res_data_all.head()

,Age,Fare,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
PassengerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,22.0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,38.0,71.2833,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,26.0,7.9250,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,35.0,53.1000,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,35.0,8.0500,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# Нормализуем данные
from sklearn.preprocessing import StandardScaler
nn_scaler = StandardScaler()
res_data_all_scaled = nn_scaler.fit_transform(res_data_all)

print(res_data_all_scaled.shape)
print(res_data_all_scaled[0])

(1309, 43)
[-0.61197171 -0.50329106 -0.57235147 -0.51808383  0.91992518 -0.74349692
  0.74349692 -1.45999279  1.76166066 -0.18206914 -0.12456285 -0.13074409
 -0.06785837 -0.08320503  0.55352256 -0.3863337  -0.3073786  -0.0784163
 -0.06785837 -0.06785837 -0.03911805 -0.03911805  0.65501092 -0.50976981
 -0.32204029  0.53937651 -0.53937651 -1.23375767  2.1378056  -0.37183447
 -0.18429662 -0.13074409 -0.13953639 -0.11123994 -0.0784163  -0.09205746
  1.23375767 -1.23375767  0.85392868 -0.22108416 -0.42215853 -0.50262469
 -0.15051993]


### Использование моделей машинного обучения для решения задачи классификации в рамках конкурса Titanic

#### Решающие деревья

In [ ]:
# Разобьем данные на обучение и валидацию
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data[:len_train_data], 
                                                    y, 
                                                    test_size=0.10, 
                                                    random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 3, 
                             min_samples_split=5, 
                             random_state=0)
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
# Оценка качества работы сети
from sklearn.metrics import accuracy_score
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict))

0.8555555555555555


In [ ]:
# Получение предсказаний для участия в конкурсе
y_compet = model.predict(all_data[len_train_data:])
print(y_compet.shape)
print(y_compet)

(418,)
[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 0 0 0 1]


##### Участие в конкурсе на kaggle.com

In [ ]:
# Участие в конкурсе
pass_id = all_data[len_train_data:].index.values
submit = pd.DataFrame({'PassengerId':pass_id,'Survived':y_compet})
#creating submission file
filename = 'Titanic_Prediction.csv'
submit.to_csv(filename,index=False)
print('Saved file: ' + filename)


Saved file: Titanic_Prediction.csv


In [ ]:
!ls

gender_submission.csv  sample_data  test.csv  Titanic_Prediction.csv  train.csv


In [ ]:
!kaggle competitions submit -c titanic -f Titanic_Prediction.csv -m "Submit DTree2.0."

100% 2.77k/2.77k [00:02<00:00, 987B/s]
Successfully submitted to Titanic: Machine Learning from Disaster

#### Случайный лес

In [ ]:
# Разобьем данные на обучение и валидацию
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data[:len_train_data], 
                                                    y, 
                                                    test_size=0.10, 
                                                    random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=1, 
                               max_depth=5,
                               n_estimators=250,  
                               max_features="auto")
model.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)

In [ ]:
# Получение предсказаний для участия в конкурсе
y_compet = model.predict(all_data[len_train_data:])
print(y_compet.shape)
print(y_compet)

##### Участие в конкурсе на kaggle.com

In [ ]:
# Участие в конкурсе
pass_id = all_data[len_train_data:].index.values
submit = pd.DataFrame({'PassengerId':pass_id,'Survived':y_compet})
#creating submission file
filename = 'Titanic_Prediction.csv'
submit.to_csv(filename,index=False)
print('Saved file: ' + filename)


In [ ]:
!ls

In [ ]:
!kaggle competitions submit -c titanic -f Titanic_Prediction.csv -m "Submit RFC."

#### Градиентный бустинг

In [ ]:
# Разобьем данные на обучение и валидацию
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data[:len_train_data], 
                                                    y, 
                                                    test_size=0.10, 
                                                    random_state=42)

In [ ]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.0001)

model.fit(X_train, y_train)
model.score(X_test,y_test)

In [ ]:
# Получение предсказаний для участия в конкурсе
y_compet = model.predict(all_data[len_train_data:])
print(y_compet.shape)
print(y_compet)

##### Участие в конкурсе на kaggle.com

In [ ]:
# Участие в конкурсе
pass_id = all_data[len_train_data:].index.values
submit = pd.DataFrame({'PassengerId':pass_id,'Survived':y_compet})
#creating submission file
filename = 'Titanic_Prediction.csv'
submit.to_csv(filename,index=False)
print('Saved file: ' + filename)


In [ ]:
!ls

In [ ]:
!kaggle competitions submit -c titanic -f Titanic_Prediction.csv -m "Submit XGB."

#### Нейронные сети

In [ ]:
# Разобьем данные на обучение и валидацию
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(res_data_all_scaled[:len_train_data], 
                                                    y, 
                                                    test_size=0.10, 
                                                    random_state=42)

In [ ]:
# Построение нейронной сети и ее обучение
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(256, activation='tanh', input_shape=(X_train.shape[1],)))
model.add(Dense(1024, activation='tanh'))
model.add(Dense(512, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dense(256, activation='tanh'))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=300,
          validation_data=(X_test,y_test))

In [ ]:
y_compet = model.predict(res_data_all_scaled[len_train_data:])

In [ ]:
y_compet = (y_compet > 0.5).astype('int64').reshape((len(y_compet),))

##### Участие в конкурсе на kaggle.com

In [ ]:
# Участие в конкурсе
pass_id = all_data[len_train_data:].index.values
submit = pd.DataFrame({'PassengerId':pass_id,'Survived':y_compet})
#creating submission file
filename = 'Titanic_Prediction.csv'
submit.to_csv(filename,index=False)
print('Saved file: ' + filename)


In [ ]:
!ls

In [ ]:
!kaggle competitions submit -c titanic -f Titanic_Prediction.csv -m "Submit NN."